Кобзарь О.С. Хабибуллин Р.А.

# Графики через plotly


Пару наборов шаблонизированных графиков.

Похоже на plotly_express

Пример из данных по учебному месторождению №8

Можно все запустить, должно отработать на примерах

Формат - квадратная таблица, первая колонка - время.

Еще формат распространненный - 3 колонки - время, название параметра, значения, парсер для него тоже тут есть

Импорт модулей. preproc_tool - для предобработки данных, в том числе переименования параметров, plotly_workflow для построения графиков

In [ ]:
import sys
sys.path.append('../')
sys.path.append('../../')
import sandbox.uTools.preproc_p.preproc_tool as preproc_tool
import pandas as pd
import sandbox.uTools.plot_workflow.plotly_workflow as plotly_workflow
import os

Вид формата данных. Все .csv, первая колонка 'Время'

In [ ]:
file = pd.read_csv('data/plotly_workflow_example_data.csv', index_col = [0], parse_dates=True,dayfirst=True )
file = file.sort_index()
file.head()

Список событий - одно событие как левая и правая граница - для выделения на графике

In [ ]:
event_borders = [[file.index[0], file.index[12]], 
                 [file.index[-1], file.index[-2]]]

Графики строятся через банчи. на одном графике может быть несколько кривых по формату  {'Название графика №1': ['Название параметра 1','Название параметра 2' ]}. Здесь функция частного случая для всех графиков в df.

In [ ]:
def create_banches_for_report(df, parameters_list):
    all_banches = []
    for i in parameters_list:
        if i in df.columns:
            test_df = df.dropna(subset = [i])
            if test_df.shape[0] > 0:
                one_banch = {i:[i]}
                all_banches.append(one_banch)
    return all_banches

Список параметров (столбцов) для графика.

In [ ]:
parameters_list = list(file.columns)[0:10]

Банчи

In [ ]:
all_banches = create_banches_for_report(file, parameters_list)

Построение графиков для df с событиями

In [ ]:
plotly_workflow.create_report_html(file, all_banches, 'test.html',
                           auto_open = True, layout_height=250, vertical_spacing=0.025, borders=event_borders)

Для более точного построения нужно переименовать столбцы и привести названия к единому формату

In [ ]:
gn = preproc_tool.GlobalNames()

In [ ]:
file = preproc_tool.rename_columns_by_dict(file)

Теперь можно не ошибаться с названиями параметров, а вызывать нужный параметр через переменные

In [ ]:
#parameters_list = [gn.freq_hz,  gn.work_status_number,gn.i_a_motor_a,  gn.motor_load_perc,
#                   gn.p_intake_atm, gn.t_intake_c, gn.t_motor_c, gn.active_power_kwt, gn.cos_phi_d, gn.vibration_xy_msec2, 
#                   gn.vibration_z_msec2, 'Вибрация']

In [ ]:
for i in file.columns:
    print(i)

In [ ]:
parameters_list = ['OIL_RATE', 'LIQ_RATE', 'WATER_CUT']

In [ ]:
all_banches = create_banches_for_report(file, parameters_list)

In [ ]:
plotly_workflow.create_report_html(file, all_banches, 'test.html',
                           auto_open = True, layout_height=250, vertical_spacing=0.025, borders=event_borders)

Дополнительно о работе с банчами

In [ ]:
all_banches = [ {'Дебиты': ['OIL_RATE', 'LIQ_RATE']}, 
               {'Обводненность и ГФ': [ 'WATER_CUT', 'GAS_FACTOR']},]

In [ ]:
plotly_workflow.create_report_html(file, all_banches, 'test.html',
                           auto_open = True, layout_height=250, vertical_spacing = None, borders=event_borders)

Быстрое построение графиков

Все графики на двух сабплотах, можно выбирать вручную.

In [ ]:
traces = plotly_workflow.create_traces_list_for_all_columms(file, 
                                                         'lines+markers',
                                                         use_gl = True)
plotly_workflow.plot_subplots(traces,  "test.html",
                      two_equal_subplots = True, auto_open = True)

Графики в ноутбуке

In [ ]:
traces = plotly_workflow.create_traces_list_for_all_columms(file[file.columns[0:5]], chosen_mode='lines+markers',
    use_gl=True)


In [ ]:
plotly_workflow.plot_func(traces, iplot_option = True, plot_title_str='Check')

Поиск всех файликов в папке, для которых нужно построить графики

In [ ]:
files = preproc_tool.find_full_path_by_pattern(os.getcwd(), '*esp_ongkm\\*', '.xlsx')

Тут формат нестандартный, приведем к квадратному (шахматкоподобному) виду. Для всех файликов график в виде раздельных subplots

In [ ]:
for i in files:
    print(i)
    file = preproc_tool.parse_standart_file(i, time_column_name = 'DT', 
                                            param_name_column_name='LONG_NAME',
                                            value_column_name='VAL')
    all_banches = create_banches_for_report(file, list(file.columns))
    html_name = i.replace('xlsx', 'html')
    plotly_workflow.create_report_html(file, all_banches, html_name,
                           auto_open = True, layout_height=250, vertical_spacing=None, borders=[])
    
    
    

Тут в виде совмещенных двух сабплотах

In [ ]:
for i in files:
    file = preproc_tool.parse_standart_file(i, time_column_name = 'DT', 
                                            param_name_column_name='LONG_NAME',
                                            value_column_name='VAL')
    all_banches = create_banches_for_report(file, list(file.columns))
    html_name = i.replace('.xlsx', '_subplots.html')
    traces = plotly_workflow.create_traces_list_for_all_columms(file, 
                                                         'lines+markers',
                                                         use_gl = True)
    plotly_workflow.plot_subplots(traces,  html_name,
                          two_equal_subplots = True, auto_open = False)